# W02 - General Data Preprocessing
In this hands-on session, we will learn some basic data pre-processings, which include:
1. duplicated data handling,
2. missing value handling,
3. data transformation (scalling and converting 'categorical data' to 'numerical data'),
4. (additional) strength indicator of a relationship between two variables.

## Read dataset "raw-flight-data.csv"
1. Download the dataset "purchases.csv" in the course portal "IF4041 Ilmu Data dan Penggalian Data".

In [1]:
import pandas as pd # import pandas library


# read csv file into 'df' dataframe

df = pd.read_csv('./dataset/raw-flight-data.csv', sep = ",") # adjust with your own csv file path 

# print some data rows
df.head()

,DayofMonth,DayOfWeek,Carrier,OriginAirportID,DestAirportID,DepDelay,ArrDelay
0,19,5,DL,11433,13303,-3.0,1.0
1,19,5,DL,14869,12478,0.0,-8.0
2,19,5,DL,14057,14869,-4.0,-15.0
3,19,5,DL,15016,11433,28.0,24.0
4,19,5,DL,11193,12892,-6.0,-11.0


## Question 01 (Q01)
The given dataset is still a 'raw dataset' with duplicated data and missing values,<br>
1. In Q01, please delete the duplicated data and return a dataframe with no duplicated data. **Hint:** use pandas API to handle the duplicated data,<br>
2. Print how many duplicated data. **Hint:** calculate the difference of row numbers, before and after we delete the duplicated data rows.

In [2]:
# write you own code for Q01 here
data_series = df.drop_duplicates()

count = len(df) - len(data_series)
print(count)

22435


## Question 02 (Q02)
In Q02, we will handle data rows with missing values. **Note:** in the given dataset, the missing values are only in column 'DepDelay' and 'ArrDelay'.
1. First and naive approach is by deleting the data rows with missing value. From data in Q01, use pandas API to remove data rows with 'missing value', with specifications: (i) column subset to be checked = 'DepDelay' and 'ArrDelay', (ii) delete the data rows with **at least one missing value** in the given subset in (i),
2. Print how many missing rows from (i),
3. Another approach to handle missing values is by filling those missing values by their corresponding mean values, most frequent values, interpolated values, etc. In this Q02, fill the missing values in the columns of 'DepDelay' and 'ArrDelay' by their corresponding mean values.
4. Perform 'drop missing value', similar with (1), to the result of (3). Calculate the difference of row numbers before and after. You should get '0' for this. 
5. In your opinion, what is the difference between those two techniques (delete missing values vs filling with mean values)? Which one do you prefer? Please explain your argument.

In [7]:
# write you own code for Q02 here
def drop_missing_values(data_series):
    return data_series.dropna(subset=['DepDelay', 'ArrDelay'])

def count_missing_rows(data_series, cleaned_data):
    return len(data_series) - len(cleaned_data)

# (i) Drop missing values
cleaned_data = drop_missing_values(data_series)
print('Dropped rows for (i) approach: ')
print(count_missing_rows(data_series, cleaned_data))

# (ii) Filling missing values
filled_df = data_series.fillna(data_series['DepDelay'].mean(), inplace=False)
filled_df = filled_df.fillna(data_series['ArrDelay'].mean(), inplace=False)

cleaned_data = drop_missing_values(filled_df)

print('Dropped rows for (ii) approach: ')
print(count_missing_rows(filled_df, cleaned_data))

Dropped rows for (i) approach: 
23798
Dropped rows for (ii) approach: 
0


Write your answer here for Q02.5 (**Note**: in case you are not familiar, this is a 'markdown' environment) <br>

When we drop missing values, we will exclude 5235 data (**0.19%**). Meanwhile when we fill the missing values with mean, we still use 100% dataset. I prefer to fill the rows with mean because 0.19% of population is huge and filling the column with average values is good enough to represent the missing values.

## Question 03 (Q03)
There is a 'Carrier' column in the given dataset, which is a "categorical variable". 
1. To build a learning model, what we need is numerical data so that our data can be processed by the learning model. Please convert "categorical valriable" in the column of "Carrier" to "numerical variable". E.g., given [A, B, A, C], we expect to get [0, 1, 0, 2] or [1, 2, 1, 3]. 

In [8]:
# write you own code for Q03 here
filled_df['Carrier'] = filled_df['Carrier'].astype('category')
categories = filled_df.select_dtypes(['category']).columns

filled_df[categories] = filled_df[categories].apply(lambda x: x.cat.codes)
filled_df.head()

,DayofMonth,DayOfWeek,Carrier,OriginAirportID,DestAirportID,DepDelay,ArrDelay
0,19,5,4,11433,13303,-3.0,1.0
1,19,5,4,14869,12478,0.0,-8.0
2,19,5,4,14057,14869,-4.0,-15.0
3,19,5,4,15016,11433,28.0,24.0
4,19,5,4,11193,12892,-6.0,-11.0


## Question 04 (Q04)
In some learning methods, we may perform data transformations so that we can expect a better performance (accuracy). One of the popular data transformation is **scalling**. Please do scalling the data column of "DepDelay" to [-1, 1]. <br>
**Hint:** In this hands-on session, don't use libary to do scalling, we expect you use scalling formula of:<br><br>
$$ \widehat{X} = \frac{(X-X_{min}) \times (BA - BB)}{X_{max}-X_{min}} + BB,$$ <br>
Where:<br>
$
\begin{align}
    \widehat{X} &= \text{scalled value}\\
    X &= \text{value being scalled}\\
    X_{max}, X_{min} &= \text{max of } X \text{, min of }X\\
    BA &= \text{batas atas, untuk kasus kita adalah 1}\\
    BB &= \text{batas bawah, untuk kasus kita adalah -1}\\
\end{align}
$

In [11]:
# write you own code for Q04 here
max_value = filled_df.loc[:, 'DepDelay'].max()
min_value = filled_df.loc[:, 'ArrDelay'].min()

def scalling(x):
    return (((x - min_value)*2/(max_value - min_value)) - 1)

scalled_df = filled_df
scalled_df['DepDelay'] = filled_df['DepDelay'].apply(scalling)
scalled_df.head()

,DayofMonth,DayOfWeek,Carrier,OriginAirportID,DestAirportID,DepDelay,ArrDelay
0,19,5,4,11433,13303,0.959853,1.0
1,19,5,4,14869,12478,0.959917,-8.0
2,19,5,4,14057,14869,0.959831,-15.0
3,19,5,4,15016,11433,0.960520,24.0
4,19,5,4,11193,12892,0.959788,-11.0


## Question 05 (Q05)
The strength of the relationship between two variables can be indicated by correlation value.
1. Please calculate correlation value between 'DepDelay' and 'ArrDelay' columns. 
2. Based on the result of (1), what can you infer?

In [12]:
# write you own code for Q05 here
scalled_df['DepDelay'].corr(scalled_df['ArrDelay'])

0.93944401733278371

`DepDelay` and `ArrDelay` has strong correlation (near to one)